In [ ]:
import findspark
findspark.init()  # Automatically finds and sets SPARK_HOME

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("MyApp") \
    .master("local[*]") \
    .getOrCreate()

# Check Spark version
print("Spark version:", spark.version)
sc=spark.sparkContext

Spark version: 3.5.5


# Customers Data

In [ ]:
df=spark.read.csv("Customer_table.csv",header=True,inferSchema=True,mode="DROPMALFORMED")
df.show(10)

+---------------+---------------+-----------+-------------------+-------------+-------+---------------+---------------+-------------+---------+------------+---------------+--------------+----------------------+---------+----------------+------------+--------------+----------------+--------+-----------+
|Pickup DateTime|  Drop DateTime|Driver Name|Driver Phone Number|Trip Distance|Trip ID|Pickup Location|  Drop Location|Trip Duration|Trip Fare|Vehicle Type|   Trip Purpose|Passenger Name|Passenger Phone Number|Scheduled|Passenger Rating|Driver Rated|Payment Method|Payment Discount|Net Fare|Trip Status|
+---------------+---------------+-----------+-------------------+-------------+-------+---------------+---------------+-------------+---------+------------+---------------+--------------+----------------------+---------+----------------+------------+--------------+----------------+--------+-----------+
| 1/1/2016 21:11| 1/1/2016 21:17|     Almire|         9298608912|           21| 318886| 

In [ ]:
df.printSchema()

root
 |-- Pickup DateTime: string (nullable = true)
 |-- Drop DateTime: string (nullable = true)
 |-- Driver Name: string (nullable = true)
 |-- Driver Phone Number: long (nullable = true)
 |-- Trip Distance: integer (nullable = true)
 |-- Trip ID: integer (nullable = true)
 |-- Pickup Location: string (nullable = true)
 |-- Drop Location: string (nullable = true)
 |-- Trip Duration: double (nullable = true)
 |-- Trip Fare: double (nullable = true)
 |-- Vehicle Type: string (nullable = true)
 |-- Trip Purpose: string (nullable = true)
 |-- Passenger Name: string (nullable = true)
 |-- Passenger Phone Number: long (nullable = true)
 |-- Scheduled: string (nullable = true)
 |-- Passenger Rating: double (nullable = true)
 |-- Driver Rated: string (nullable = true)
 |-- Payment Method: string (nullable = true)
 |-- Payment Discount: string (nullable = true)
 |-- Net Fare: double (nullable = true)
 |-- Trip Status: string (nullable = true)



# Admin Data

In [ ]:
df3=spark.read.csv("Admin_data.csv",header=True,inferSchema=True,mode="DROPMALFORMED")
df3.show(10)

+---------------+---------------+-------+----------+---+---------+---------------+-----------+---------------+-----+-----------+-------------+---------------+-----+------+--------+--------+-----+-------------+---------+---------+---------+----------+----------+-----------+--------------+---------------+---------+----------+------------+--------+----------+---------+
|     Start_time|       End_time|   Name|    Mobile|Age|Pin-Codes|         Source|Vaccine_cus|    Destination|Miles|Est_Costing|Ride_category|        Purpose| temp|clouds|pressure|humidity| wind|accquire_vehi|free_vehi|Lattitute|Longitude|locationID|rating_cus|Riders_Name|Riders_contact|Trusted_Contact|Rating_RI|Vaccine_Ri|Payment_mode|Discount|Final_cost|   Status|
+---------------+---------------+-------+----------+---+---------+---------------+-----------+---------------+-----+-----------+-------------+---------------+-----+------+--------+--------+-----+-------------+---------+---------+---------+----------+----------+-

In [ ]:
df3.printSchema()

root
 |-- Start_time: string (nullable = true)
 |-- End_time: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Mobile: long (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Pin-Codes: integer (nullable = true)
 |-- Source: string (nullable = true)
 |-- Vaccine_cus: string (nullable = true)
 |-- Destination: string (nullable = true)
 |-- Miles: double (nullable = true)
 |-- Est_Costing: double (nullable = true)
 |-- Ride_category: string (nullable = true)
 |-- Purpose: string (nullable = true)
 |-- temp: double (nullable = true)
 |-- clouds: double (nullable = true)
 |-- pressure: double (nullable = true)
 |-- humidity: double (nullable = true)
 |-- wind: double (nullable = true)
 |-- accquire_vehi: integer (nullable = true)
 |-- free_vehi: string (nullable = true)
 |-- Lattitute: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- locationID: integer (nullable = true)
 |-- rating_cus: integer (nullable = true)
 |-- Riders_Name: string (null

In [ ]:
df3=spark.read.csv("Admin_data.csv",header=True,inferSchema=True,mode="DROPMALFORMED")
df3.show(5)

+--------------+--------------+-------+----------+---+---------+---------------+-----------+---------------+-----+-----------+-------------+---------------+-----+------+--------+--------+-----+-------------+---------+---------+---------+----------+----------+-----------+--------------+---------------+---------+----------+------------+--------+----------+---------+
|    Start_time|      End_time|   Name|    Mobile|Age|Pin-Codes|         Source|Vaccine_cus|    Destination|Miles|Est_Costing|Ride_category|        Purpose| temp|clouds|pressure|humidity| wind|accquire_vehi|free_vehi|Lattitute|Longitude|locationID|rating_cus|Riders_Name|Riders_contact|Trusted_Contact|Rating_RI|Vaccine_Ri|Payment_mode|Discount|Final_cost|   Status|
+--------------+--------------+-------+----------+---+---------+---------------+-----------+---------------+-----+-----------+-------------+---------------+-----+------+--------+--------+-----+-------------+---------+---------+---------+----------+----------+-------

In [ ]:
df3.printSchema()

root
 |-- Start_time: string (nullable = true)
 |-- End_time: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Mobile: long (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Pin-Codes: integer (nullable = true)
 |-- Source: string (nullable = true)
 |-- Vaccine_cus: string (nullable = true)
 |-- Destination: string (nullable = true)
 |-- Miles: double (nullable = true)
 |-- Est_Costing: double (nullable = true)
 |-- Ride_category: string (nullable = true)
 |-- Purpose: string (nullable = true)
 |-- temp: double (nullable = true)
 |-- clouds: double (nullable = true)
 |-- pressure: double (nullable = true)
 |-- humidity: double (nullable = true)
 |-- wind: double (nullable = true)
 |-- accquire_vehi: integer (nullable = true)
 |-- free_vehi: string (nullable = true)
 |-- Lattitute: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- locationID: integer (nullable = true)
 |-- rating_cus: integer (nullable = true)
 |-- Riders_Name: string (null

# 1.How many no of customers take trip from same location.

In [ ]:
from pyspark.sql.functions import count
df.groupBy("Pickup Location").agg(count("Passenger Name").alias("customer_count")).orderBy('customer_count',ascending=False).show()

+-----------------+--------------+
|  Pickup Location|customer_count|
+-----------------+--------------+
|      Fort Pierce|           108|
|          Midtown|            78|
|  West Palm Beach|            54|
|             Cary|            52|
|  Lower Manhattan|            26|
|     Midtown East|            26|
|Flatiron District|            26|
|      East Harlem|            26|
|    Hudson Square|            26|
|          Jamaica|            26|
|         New York|            26|
|         Elmhurst|            26|
+-----------------+--------------+



# 2.what is priority for each ride category from each location.

# 3.what are the longest locations of customer travelled.

In [ ]:
df.orderBy('Trip Distance',ascending=False).select('Passenger Name','Pickup Location','Drop Location','Trip Distance').limit(10).show()

+--------------+---------------+-------------+-------------+
|Passenger Name|Pickup Location|Drop Location|Trip Distance|
+--------------+---------------+-------------+-------------+
|      Virginie|       Elmhurst|         Cary|           80|
|      Charlena|   Midtown East|       Durham|           80|
|         Price|    East Harlem|  Whitebridge|           80|
|      Darlleen|    Fort Pierce|         Cary|           80|
|        Haskel|           Cary|  Whitebridge|           80|
|         Jacky|West Palm Beach|      Houston|           80|
|      Collette|    Fort Pierce|   Tanglewood|           80|
|          Moss|Lower Manhattan|  Morrisville|           80|
|          Elsy|       New York|         Cary|           79|
|         Daron|    Fort Pierce|         Cary|           79|
+--------------+---------------+-------------+-------------+



# 4. Drivers who completed ride with non -vaccinated customers.

In [ ]:
df3.filter(df3.Vaccine_cus=='NO').select("Name").show()

+-------+
|   Name|
+-------+
| Correy|
| Gabbey|
|Mathian|
|Huntlee|
|   Clim|
|Geordie|
| Horten|
|    Ray|
|  Boyce|
|   Berk|
| Aubert|
|Stavros|
| Norene|
|  Stace|
| Stefan|
|Cassius|
| Crysta|
|  Chadd|
|    Val|
|  Glori|
+-------+
only showing top 20 rows



# 5.How many vaccinated customers have travelled

In [ ]:
print(f"{df3.filter(df3.Vaccine_cus=='YES').count()} vaccinated customers have travelled")

400 vaccinated customers have travelled


# 6. .Customers who completed ride with non -vaccinated Drivers

In [ ]:
df3.filter(df3.Vaccine_Ri=='NO').select("Riders_Name").show()

+-----------+
|Riders_Name|
+-----------+
|      Myrle|
|    Ellette|
|     Mikkel|
|       Abbi|
|       Dora|
|     Pattin|
|        Kip|
|     Haskel|
|      Cindi|
|    Yasmeen|
|       Burk|
|      Natty|
|       Kain|
|        Pen|
|      Sandy|
|   Kimberly|
|        Tim|
|    Carolus|
|      Marta|
|    Romonda|
+-----------+
only showing top 20 rows



# 7.who is the customer completed highest no of rides.

In [ ]:
from pyspark.sql.functions import count
df.groupBy("Passenger Name").agg(count("Trip ID").alias("trip_count")).orderBy('trip_count',ascending=False).limit(1).show()

+--------------+----------+
|Passenger Name|trip_count|
+--------------+----------+
|          Cher|        10|
+--------------+----------+



# 8.who is the driver completed highest no of rides.

In [ ]:
df3.groupBy('Name').count().orderBy('count',ascending=False).limit(11).show()

+---------+-----+
|     Name|count|
+---------+-----+
|  Mathian|    2|
|   Bidget|    2|
|   Teresa|    2|
|    Mayne|    2|
|Westbrook|    2|
|    Debor|    2|
|   Raeann|    2|
|      Gus|    2|
|  Othello|    2|
|  Frazier|    2|
| Roderigo|    2|
+---------+-----+



# 9.what are first 10 age groups which uses uber services mostly.

In [ ]:
from pyspark.sql.functions import count
df3.groupBy("Age").agg(count('Riders_Name').alias('customer_age_count')).orderBy('customer_age_count',ascending=False).limit(10).show()

+---+------------------+
|Age|customer_age_count|
+---+------------------+
| 69|                16|
| 15|                15|
| 32|                15|
| 28|                12|
| 78|                11|
| 43|                11|
| 49|                11|
| 21|                11|
| 23|                10|
| 44|                10|
+---+------------------+



# 10.what is the count of different destination locations from same start location and also completed ride

In [ ]:
from pyspark.sql.functions import countDistinct

df.filter(df.Source=='Arrived').groupBy()